<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    ReduceLROnPlateau in Keras: Adaptive Learning Rate Management
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Learning Rate Adaptation with ReduceLROnPlateau
#         
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Create synthetic data for learning rate adaptation
# 2. Implement ReduceLROnPlateau callback
# 3. Monitor learning rate changes during training
# 4. Visualize the impact of learning rate reduction
# 5. Compare training with and without adaptive learning rates
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2025
# =======================================================================

In [2]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import os  
import shutil
from datetime import datetime

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# import tensorflow
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2024-12-30

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
sklearn   : 1.5.1
pandas    : 2.2.2
numpy     : 1.26.4
keras     : 3.6.0
tensorflow: 2.16.2
seaborn   : 0.13.2
matplotlib: 3.9.2



In [5]:
# =========================================================================
# utility function to clean up a directory
# This function is useful for me (the instructor) while preparing the 
# the educational content. It is also useful for you (the student)
# while you are playing around with the code
# =========================================================================
def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.

    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")

## Overview

The `ReduceLROnPlateau` callback in Keras is an adaptive learning rate mechanism that reduces the learning rate when a monitored metric has stopped improving. This is particularly useful for:

 - **Fine-tuning models**
 - **Navigating training plateaus**
 - **Achieving better model convergence**
 - **Optimizing final model performance**

## Key Parameters

```python
ReduceLROnPlateau(
    monitor='val_loss',      # Metric to monitor
    factor=0.1,              # Factor by which to reduce LR
    patience=10,             # Number of epochs with no improvement
    verbose=0,               # Logging mode
    mode='auto',             # Direction of optimization
    min_delta=0.0001,        # Minimum change to qualify as improvement
    cooldown=0,              # Epochs to wait before resuming monitoring
    min_lr=0,                # Lower bound on learning rate
)
```

### Parameter Details

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| `monitor` | Metric to track | 'val_loss', 'val_accuracy' |
| `factor` | Reduction factor | 0.1 to 0.5 |
| `patience` | Epochs to wait | 5 to 10 |
| `min_delta` | Minimum improvement | 1e-4 to 1e-2 |
| `cooldown` | Recovery period | 2 to 5 epochs |
| `min_lr` | Minimum LR | 1e-6 to 1e-4 |

## Implementation Example

```python
def create_reduce_lr_callback():
    return ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,          # Reduce by 20%
        patience=5,
        min_delta=0.001,
        cooldown=2,
        min_lr=1e-6,
        verbose=1
    )

# Usage with model.fit()
history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[create_reduce_lr_callback()],
    verbose=1
)
```

## Working Mechanism

1. **Monitoring Phase**
   - Tracks specified metric each epoch
   - Compares with best recorded value

2. **Decision Phase**

```plaintext
If no improvement for 'patience' epochs:
   new_lr = max(current_lr * factor, min_lr)
   Update learning rate
   Enter cooldown period
```

3. **Cooldown Phase**
   - Temporarily suspends monitoring
   - Allows model to adapt to new rate

## Best Practices

### 1. Monitoring Selection
- Use `val_loss` for general cases
- Consider `val_accuracy` for classification
- Custom metrics should be smooth and stable

### 2. Parameter Tuning

```python
# Conservative Setup
reduce_lr = ReduceLROnPlateau(
    factor=0.5,
    patience=10,
    min_delta=0.01
)

# Aggressive Setup
reduce_lr = ReduceLROnPlateau(
    factor=0.2,
    patience=5,
    min_delta=0.001
)
```

### 3. Integration with Other Callbacks

```python
callbacks = [
    ReduceLROnPlateau(...),
    EarlyStopping(...),
    ModelCheckpoint(...)
]
```

## Common Patterns

1. **Early Training**
   - Higher learning rates
   - Shorter patience
   - Larger reduction factor

2. **Late Training**
   - Lower learning rates
   - Longer patience
   - Smaller reduction factor

## Visualization Code

```python
def plot_lr_history(history):
    """Plot learning rate changes over training"""
    plt.figure(figsize=(10, 6))
    
    # Plot training metrics
    plt.plot(history.history['val_loss'], label='Validation Loss')
    
    # Add LR change markers
    for epoch in detect_lr_changes(history):
        plt.axvline(x=epoch, color='r', linestyle='--', alpha=0.5)
    
    plt.title('Training Progress with LR Changes')
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()
```

## Troubleshooting

### Common Issues and Solutions

1. **Too Frequent Reductions**
   - Increase `patience`
   - Increase `min_delta`
   - Add `cooldown` period

2. **No Reductions**
   - Decrease `patience`
   - Decrease `min_delta`
   - Check `monitor` metric

3. **Premature Minimum LR**
   - Increase `factor`
   - Decrease `min_lr`
   - Adjust initial learning rate

## Advanced Usage

### Custom Reduction Schedules

```python
class CustomReduceLR(ReduceLROnPlateau):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.custom_factors = [0.5, 0.2, 0.1]
        self.factor_index = 0
    
    def get_reduced_lr(self):
        if self.factor_index < len(self.custom_factors):
            factor = self.custom_factors[self.factor_index]
            self.factor_index += 1
            return self.current_lr * factor
        return self.current_lr * self.factor
```

### Monitoring Multiple Metrics

```python
class MultiMetricReduceLR(ReduceLROnPlateau):
    def __init__(self, monitors=['val_loss', 'val_accuracy'], **kwargs):
        super().__init__(**kwargs)
        self.monitors = monitors
    
    def get_monitor_value(self, logs):
        return max(logs.get(m) for m in self.monitors)
```